# Conformalized quantile regression (CQR): Real data experiment

In this tutorial we will load a real dataset and construct prediction intervals using CQR [1].

[1] Yaniv Romano, Evan Patterson, and Emmanuel J. Candes, “Conformalized quantile regression.” 2019.

In [54]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GitHub/dissertation

!git config --global user.email "sann7266@ox.ac.uk"
!git config --global user.name "Roel Hulsman"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/dissertation


In [55]:
!pip install scikit-garden
!pip install scikit-learn==0.21.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



## Real Data Case Study

We start by importing several libraries, loading the real dataset and standardize its features and response. 

In [70]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
np.warnings.filterwarnings('ignore')
import pandas as pd
import torch
import timeit
import random

from cqr import helper
from datasets import datasets
from nonconformist.nc import RegressorNc
from nonconformist.cp import IcpRegressor
from nonconformist.nc import QuantileRegErrFunc
from nonconformist.nc import QuantileRegAsymmetricErrFunc
from sklearn.preprocessing import StandardScaler
from scipy.stats import binom

%matplotlib inline

# Seed
#seed = 23
#seed = 19
#seed = 17
#seed = 13
#seed = 11
#seed = 7
#seed = 5
#seed = 3
#seed = 2
seed = 1

# size of test set as compared to train set
test_ratio = 0.2

# save figures and runs?
save_figures = True
save_run = True

### Load Dataset Dimensions and Objective

In [71]:
np.random.seed(seed)

# name of dataset
dataset_base_path = "./datasets/"
dataset_name = "concrete"

# load the dataset
X, y = datasets.GetDataset(dataset_name, dataset_base_path)
p = X.shape[1] 

# calculate sample sizes based on test_ratio
n_train = np.int(np.floor(len(y)*(1-test_ratio)/2))
n_cal = np.int(np.floor(len(y)*(1-test_ratio)/2))
n_test = np.int(len(y)-n_train-n_cal)

# display basic information
print("Dataset: %s" % (dataset_name))
print("Dimensions: p=%d, train set (n=%d) ; calibration set (n=%d) ; test set (n=%d)" % 
      (p, n_train, n_cal, n_test))

# desired tolerance region 
eps = 0.1
delta = 0.1

# number of loops
R = 1000

# desired expected miscoverage error
alpha = (max(np.where(binom.cdf(range(n_cal), n_cal, eps)<=delta)[-1])+1)/(n_cal+1)

# low and high target quantiles
quantiles = [100*alpha/2, 100*(1-alpha/2)]

print("Tolerance Region: Epsilon = ", eps, ", Delta = ", delta)
print("Marginal Coverage: Alpha = ", alpha, ", Target Quantiles: ", quantiles)
print("Number of Splits: R = ", R)

Dataset: concrete
Dimensions: p=8, train set (n=412) ; calibration set (n=412) ; test set (n=206)
Tolerance Region: Epsilon =  0.1 , Delta =  0.1
Marginal Coverage: Alpha =  0.07990314769975787 , Target Quantiles:  [3.9951573849878934, 96.00484261501211]
Number of Splits: R =  1000


## CQR random forests

Given these two subsets, we now turn to conformalize the initial prediction interval constructed by quantile random forests [2]. Below, we set the hyper-parameters of the CQR random forests method.

[2] Meinshausen Nicolai. "Quantile regression forests." Journal of Machine Learning Research 7, no. Jun (2006): 983-999.

In [72]:
# define quantile random forests (QRF) parameters
params_qforest = dict()
params_qforest["n_estimators"] = 100 # the number of trees in the forest
params_qforest["min_samples_leaf"] = 40 # the minimum number of samples required to be at a leaf node (default)
params_qforest["max_features"] = p # the number of features to consider when looking for the best split (default)
params_qforest["CV"] = True # use cross-validation to tune the quantile levels?
params_qforest["coverage_factor"] = 0.9 # ask for smaller coverage for QRF quantiles to avoid conservative estimate
params_qforest["test_ratio"] = 0.1 # ratio of held-out data, used in cross-validation
params_qforest["random_state"] = seed # seed for splitting the data in cross-validation
params_qforest["range_vals"] = 30 # determines the lowest and highest quantile level parameters
params_qforest["num_vals"] = 10 # sweep over a grid of length num_vals when tuning QRF's quantile parameters 

### Data splitting

We begin by splitting the data into a proper training set and a calibration set. Recall that the main idea is to fit a regression model on the proper training samples, then use the residuals on a held-out validation set to quantify the uncertainty in future predictions.

In [73]:
np.random.seed(seed)

# divide the data into proper training and rest (calibration + test) set
idx = np.random.permutation(len(y))
idx_train = idx[:n_train]
idx_rest = idx[n_train:len(y)]

# divide the dataset into proper train and rest (calibration + test) data 
x_train, y_train = X[idx_train,], y[idx_train]
x_rest, y_rest = X[idx_rest,], y[idx_rest]

# reshape the data
x_train, y_train = np.asarray(x_train), np.asarray(y_train)
x_rest, y_rest = np.asarray(x_rest), np.asarray(y_rest)

# zero mean and unit variance scaling 
scalerX = StandardScaler()
scalerX = scalerX.fit(x_train)
x_train = scalerX.transform(x_train)
x_rest = scalerX.transform(x_rest)

# scale the labels by dividing each by the mean absolute response
mean_y_train = np.mean(np.abs(y_train))
y_train = np.squeeze(y_train)/mean_y_train
y_rest = np.squeeze(y_rest)/mean_y_train

### Symmetric nonconformity score 

In the following cell we run the entire CQR procudure. The class `QuantileForestRegressorAdapter` defines the underlying estimator. The class `RegressorNc` defines the CQR objecct, which uses `QuantileRegErrFunc` as the nonconformity score. The function `run_icp` fits the regression function to the proper training set, corrects (if required) the initial estimate of the prediction interval using the calibration set, and returns the conformal band. Lastly, we compute the average coverage and length on future test data using `compute_coverage`.

In [74]:
np.random.seed(seed)

start = timeit.default_timer()

# define the QRF model
quantile_estimator = helper.QuantileForestRegressorAdapter(model=None,
                                                           fit_params=None,
                                                           quantiles=quantiles,
                                                           params=params_qforest)

# define the CQR object, computing the absolute residual error of points 
# located outside the estimated QRF band 
nc = RegressorNc(quantile_estimator, QuantileRegErrFunc())

# build the split CQR object
icp = IcpRegressor(nc)

# fit the conditional quantile regression to the proper training data
icp.fit(x_train, y_train)

# compute the low and high conditional quantile estimation on calibration and test set to get indication of performance
pred = quantile_estimator.predict(x_rest)
y_lower = pred[:,0]
y_upper = pred[:,1]

# compute and display the average coverage
in_the_range = np.sum((y_rest >= y_lower) & (y_rest <= y_upper))
print("QRF: Pct in range (expecting " + str(100*(1-alpha)*params_qforest["coverage_factor"]) + "%):", in_the_range / len(y_rest) * 100)

# compute and display length of the conformal interval per each test point
length = y_upper - y_lower
print("QRF: Average length:", np.mean(length))

stop = timeit.default_timer()
print('QRF: Time: ', stop - start)

QRF: Pct in range (expecting 82.8087167070218%): 88.02588996763754
QRF: Average length: 0.8764577277590785
QRF: Time:  1.1986306470025738


In [75]:
def tol_reg_loop(
      icp = icp,
      R = 100,
      n_cal = 100,
      n_test = 100,
      alpha = 0.05,
      save_run = False,
      filename = None):

    results = np.array([[0]*3]*R, dtype=float)

    for i in range(R):

        start = timeit.default_timer()

        # calibration and test features
        idx_rest_perm = np.random.permutation(len(x_rest))
        idx_cal = idx_rest_perm[:n_cal]
        idx_test = idx_rest_perm[n_cal:(n_cal+n_test)]

        # divide the dataset into calibration and test data 
        x_cal, y_cal = x_rest[idx_cal,], y_rest[idx_cal]
        x_test, y_test = x_rest[idx_test,], y_rest[idx_test]

        # compute the absolute errors on calibration data
        icp.calibrate(x_cal, y_cal)

        # produce predictions for the test set, with confidence equal to significance
        predictions = icp.predict(x_test, significance=alpha)
        y_lower = predictions[:,0]
        y_upper = predictions[:,1]

        # compute and display the average coverage
        in_the_range = np.sum((y_test >= y_lower) & (y_test <= y_upper))
        print("CQR QRF: Pct in range (expecting " + str(100*(1-alpha)) + "%):", in_the_range / len(y_test) * 100)
        results[i][0] = in_the_range / len(y_test) * 100

        # compute length of the conformal interval per each test point
        length = y_upper - y_lower

        # compute and display the average length
        print("CQR QRF: Average length:", np.mean(length))
        results[i][1] = np.mean(length)

        stop = timeit.default_timer()
        print('CQR QRF: Runtime: ', stop - start)
        results[i][2] = stop - start
    
    results = pd.DataFrame(results,
                           columns = ['CQR Pct in range', 'CQR Avg length', 'Runtime'])
    
    if save_run and (filename is not None):
      results.to_csv(filename, index=False)

    return(results)

In [76]:
np.random.seed(seed)

run = tol_reg_loop(
      icp = icp,
      R = R,
      n_cal = n_cal,
      n_test = n_test,
      alpha = alpha,
      save_run = save_run,
      filename = "Experiments Output/concrete_cqr_s1_R1000.csv")

CQR QRF: Pct in range (expecting 92.00968523002422%): 89.80582524271846
CQR QRF: Average length: 0.9555841487076712
CQR QRF: Runtime:  0.6027517180009454
CQR QRF: Pct in range (expecting 92.00968523002422%): 94.1747572815534
CQR QRF: Average length: 0.9948105690185445
CQR QRF: Runtime:  0.5830235840003297
CQR QRF: Pct in range (expecting 92.00968523002422%): 94.1747572815534
CQR QRF: Average length: 1.0042315284359709
CQR QRF: Runtime:  0.5983082560014736
CQR QRF: Pct in range (expecting 92.00968523002422%): 88.83495145631069
CQR QRF: Average length: 0.9612510263046902
CQR QRF: Runtime:  0.5892455510002037
CQR QRF: Pct in range (expecting 92.00968523002422%): 89.32038834951457
CQR QRF: Average length: 0.9599838305311541
CQR QRF: Runtime:  0.5997303160002048
CQR QRF: Pct in range (expecting 92.00968523002422%): 94.66019417475728
CQR QRF: Average length: 1.0047020789939805
CQR QRF: Runtime:  0.5923234420006338
CQR QRF: Pct in range (expecting 92.00968523002422%): 91.74757281553399
CQR QR

As can be seen, we obtained valid coverage.

### Asymmetric nonconformity score 

The nonconformity score function `QuantileRegErrFunc` treats the left and right tails symmetrically, but if the error distribution is significantly skewed, one may choose to treat them asymmetrically. This can be done by replacing `QuantileRegErrFunc` with `QuantileRegAsymmetricErrFunc`, as implemented in the following cell.

In [ ]:
np.random.seed(seed)

start = timeit.default_timer()

# define the QRF model
quantile_estimator_asym = helper.QuantileForestRegressorAdapter(model=None,
                                                           fit_params=None,
                                                           quantiles=quantiles,
                                                           params=params_qforest)

# define the CQR object, computing the absolute residual error of points 
# located outside the estimated QRF band 
nc = RegressorNc(quantile_estimator_asym, QuantileRegAsymmetricErrFunc())

# build the split CQR object
icp_asym = IcpRegressor(nc)

# fit the conditional quantile regression to the proper training data
icp_asym.fit(x_train, y_train)

# compute the low and high conditional quantile estimation
pred = quantile_estimator_asym.predict(x_rest)
y_lower = pred[:,0]
y_upper = pred[:,1]

# compute and display the average coverage
in_the_range = np.sum((y_rest >= y_lower) & (y_rest <= y_upper))
print("QRF Asymmetric: Pct in range (expecting " + str(100*(1-alpha)*params_qforest["coverage_factor"]) + "%):", 
      in_the_range / len(y_rest) * 100)

# compute and display length of the conformal interval per each test point
length = y_upper - y_lower
print("QRF Asymmetric: Average length:", np.mean(length))

stop = timeit.default_timer()
print('QRF Asymmetric: Time: ', stop - start)

QR QRF Asymmetric: Pct in range (expecting 77.65037593984962%): 86.21553884711778
QR QRF Asymmetric: Average length: 1.3398586666427958
QR QRF Asymmetric: Time:  94.48901197799933


In [ ]:
np.random.seed(seed)

run = tol_reg_loop(
      icp = icp_asym,
      R = R,
      n_cal = n_cal,
      n_test = n_test,
      alpha = alpha,
      save_run = save_run,
      filename = "Experiments Output/community_cqr_asym_R1000.csv")

CQR QRF: Pct in range (expecting 91.35338345864662%): 91.75
CQR QRF: Average length: 1.5484802579940666
CQR QRF: Runtime:  11.559086735000164
CQR QRF: Pct in range (expecting 91.35338345864662%): 92.75
CQR QRF: Average length: 1.5339897538046472
CQR QRF: Runtime:  10.695331482000256
CQR QRF: Pct in range (expecting 91.35338345864662%): 91.75
CQR QRF: Average length: 1.5791622686331268
CQR QRF: Runtime:  11.873384799000632
CQR QRF: Pct in range (expecting 91.35338345864662%): 94.0
CQR QRF: Average length: 1.5434591758525948
CQR QRF: Runtime:  11.51321047700003
CQR QRF: Pct in range (expecting 91.35338345864662%): 92.5
CQR QRF: Average length: 1.5594466684312562
CQR QRF: Runtime:  10.540203578999353
CQR QRF: Pct in range (expecting 91.35338345864662%): 88.5
CQR QRF: Average length: 1.5056649534737376
CQR QRF: Runtime:  10.77582132599855
CQR QRF: Pct in range (expecting 91.35338345864662%): 89.25
CQR QRF: Average length: 1.4746725406331327
CQR QRF: Runtime:  10.505760668998846
CQR QRF: Pc

## Quantile Random Forest (QRF)

In [ ]:
# define quantile random forests (QRF) parameters
params_qforest["CV"] = False
params_qforest["coverage_factor"] = 1

In [ ]:
def qrf_loop(
      R = 100,
      n_train = 100,
      n_test = 100,
      alpha = 0.05,
      save_run = False,
      filename = None):

    results = np.array([[0]*3]*R, dtype=float)

    for i in range(R):

        start = timeit.default_timer()

        # divide the data into training and test set
        idx = np.random.permutation(len(y))
        idx_train = idx[:n_train]
        idx_test = idx[n_train:len(y)]

        # divide the dataset into proper train and rest (calibration + test) data 
        x_train, y_train = X[idx_train,], y[idx_train]
        x_test, y_test = X[idx_test,], y[idx_test]

        # reshape the data
        x_train, y_train = np.asarray(x_train), np.asarray(y_train)
        x_test, y_test = np.asarray(x_test), np.asarray(y_test)

        # zero mean and unit variance scaling 
        scalerX = StandardScaler()
        scalerX = scalerX.fit(x_train)
        x_train = scalerX.transform(x_train)
        x_test = scalerX.transform(x_test)

        # scale the labels by dividing each by the mean absolute response
        mean_y_train = np.mean(np.abs(y_train))
        y_train = np.squeeze(y_train)/mean_y_train
        y_test = np.squeeze(y_test)/mean_y_train

        # define the QRF model
        quantile_estimator = helper.QuantileForestRegressorAdapter(model=None,
                                                           fit_params=None,
                                                           quantiles=quantiles,
                                                           params=params_qforest)

        # fit the conditional quantile regression to the proper training data
        quantile_estimator.fit(x_train, y_train)

        # compute the low and high conditional quantile estimation
        pred = quantile_estimator.predict(x_test)
        y_lower = pred[:,0]
        y_upper = pred[:,1]

        # compute and display the average coverage
        in_the_range = np.sum((y_test >= y_lower) & (y_test <= y_upper))
        print("QRF : Pct in range (expecting " + str(100*(1-alpha)*params_qforest["coverage_factor"]) + "%):",
              in_the_range / len(y_test) * 100)
        results[i][0] = in_the_range / len(y_test) * 100

        # compute length of the conformal interval per each test point
        length = y_upper - y_lower
        print("QRF : Average length:", np.mean(length))
        results[i][1] = np.mean(length)

        stop = timeit.default_timer()
        print('QRF : Runtime: ', stop - start)
        results[i][2] = stop - start
    
    results = pd.DataFrame(results,
                           columns = ['QRF Pct in range', 'QRF Avg length', 'Runtime'])
    
    if save_run and (filename is not None):
      results.to_csv(filename, index=False)

    return(results)

In [ ]:
np.random.seed(seed)

run = qrf_loop(
      R = 100,
      n_train = n_train+n_cal,
      n_test = n_test,
      alpha = alpha,
      save_run = save_run,
      filename = "Experiments Output/concrete_qrf_s23_R100.csv")

QRF : Pct in range (expecting 92.00968523002422%): 90.77669902912622
QRF : Average length: 0.5024086797521211
QRF : Runtime:  15.788865305999934
QRF : Pct in range (expecting 92.00968523002422%): 91.2621359223301
QRF : Average length: 0.5167341856480078
QRF : Runtime:  15.603620496000076
QRF : Pct in range (expecting 92.00968523002422%): 95.14563106796116
QRF : Average length: 0.5153087949984653
QRF : Runtime:  15.65429758100072
QRF : Pct in range (expecting 92.00968523002422%): 92.23300970873787
QRF : Average length: 0.5206284022753981
QRF : Runtime:  15.833607390999532
QRF : Pct in range (expecting 92.00968523002422%): 94.1747572815534
QRF : Average length: 0.5291831040530275
QRF : Runtime:  15.573129361999236
QRF : Pct in range (expecting 92.00968523002422%): 92.71844660194175
QRF : Average length: 0.481689082412471
QRF : Runtime:  15.551188301000366
QRF : Pct in range (expecting 92.00968523002422%): 91.2621359223301
QRF : Average length: 0.5150645068135672
QRF : Runtime:  15.749445